In [1]:
pip install dash-bootstrap-components pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
### Importing liabraries
import dash
import dash_bootstrap_components as dbc
from dash import html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
#import recco
#import moods
import streamlit as st

## Loading our data

Author_sorted = pd.read_csv('Author_data.csv')

Age_sorted = pd.read_csv('Age_data.csv')

## Putting in my Age recommendation function

def author_age_based(title,Age_group, number,Author_sorted=Author_sorted):
    
  
      
      Author_df = Author_sorted.loc[Author_sorted['Title']==title]['Author'].unique()[0]
        
      Author_by_title= Author_sorted.loc[(Author_sorted['Author']==Author_df)].sort_values(by='weighted_average',ascending=False)
        
      Age_df = Age_sorted.loc[Age_sorted['Age_group']==Age_group]['Author'].unique()
      
      title_by_age = Age_sorted[Age_sorted['Author'].isin(Age_df)].sort_values(by='weighted_average',ascending = False)
        
       # Exclude the original title from the recommendations
      recommendations = title_by_age[title_by_age['Title'] != title]  
        
        
      top_recommendations = recommendations.head(number)
      
      print(f'For {Age_group} age demographic')  
      print(f'The author of the book {title} is {Author_df}\n')
      print(f'Here are the top {number} books from the same author\n')
      top_recommendations = top_recommendations[['Title', 'Author', 'weighted_average', 'Age_group']]
      return(top_recommendations)


## Initiating app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

## Setting the app layout


app.layout = dbc.Container(
    [
        html.H1("Book Recommendation App", className="text-center mb-4"),  # Title
        html.Img(src='Books_HD_(8314929977).jpg', style={'width': '100%', 'max-width': '400px', 'margin': 'auto', 'display': 'block'}),  # Image  # This set the title at the centre
        
        dbc.Row(
            dbc.Col(
                dcc.Dropdown(
                    id='Select book title',
                    options= [{'label': book, 'value': book} for book in Author_sorted['Title'].unique()],
                    style={'width': '100%'}
                ),
                width={"size": 7},
            ),
            justify="start",
        ),
        dbc.Row(
            dbc.Col(
                dcc.Dropdown(
                    id='Select age group',
                    options=[{'label': age, 'value': age} for age in Age_sorted['Age_group'].unique()], # Putting age in drop down menu
                    placeholder="Your Age Group",
                    style={'width': '100%'}
                ),
                width={"size": 6},
            ),
            justify="start",
        ),
        dbc.Row(
            dbc.Col(
                dcc.Input(
                    id='num-recommendations-input',
                    type='number',
                    placeholder='Number of recommendations',
                    style={'width': '100%'}
                ),
                width={"size": 6},
            ),
            justify="start",
        ),
        dbc.Row(
            dbc.Col(
                html.Button('Get Recommendations', id='submit-button', n_clicks=0, className='btn btn-primary'),
                width={"size": 6, "offset": 3},
                style={'textAlign': 'center', 'marginTop': '20px'}
            ),
            justify="center",
        ),
        dbc.Row(
            dbc.Col(
                html.Div(id='recommendation-output'),
                width={"size": 6, "offset": 3},
            ),
            justify="center",
        ),
    ],
    fluid=True,
)

# Callback to update the recommendations based on user input
@app.callback(
    Output('recommendation-output', 'children'),
    Input('submit-button', 'n_clicks'),
    [Input('Select book title', 'value'),
     Input('Select age group', 'value'),
     Input('num-recommendations-input', 'value')]
)
def update_recommendations(n_clicks, title, age_group, num_recommendations):
    if n_clicks == 0:
        return html.P("Enter details and click the button to get recommendations.")
    
    if not title or not age_group or not num_recommendations:
        return html.P("Please provide all the required inputs.")
    
    # Getting author and displaying additional information
    author_df = Author_sorted.loc[Author_sorted['Title'] == title]['Author'].unique()[0]
    additional_info = html.P(f"For {age_group} age demographic, The author of the book {title} is {author_df}. Here are the top {num_recommendations} books from the same author:")
    
    recommendations = author_age_based(title, age_group, num_recommendations)
    if recommendations.empty:
        return html.Div([additional_info, html.P("Huh, No recommendations found! try again..")])
    
    return html.Div([additional_info, html.Ul([html.Li(f"{book['Title']} by {book['Author']}") for book in recommendations.to_dict('records')])])
# Running the app
if __name__ == "__main__":
    app.run_server(debug=True)

C:\Users\Guest1\AppData\Local\Temp\ipykernel_25112\2692338580.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
